In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math 
from dataclasses import dataclass
from tokenizers import Tokenizer
from pathlib import Path
from transformers import AutoModelForCausalLM,AutoTokenizer
import wandb



In [7]:
##parameters
BATCH_SIZE = 128
beta = 2
lr = 5e-5
gamma = 1.2
 

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [64]:
class SimplePO(nn.Module):
    def __init__(self,policy_model,gamma,beta,tokenizer):
        super().__init__()
        self.policy_model = policy_model
        self.gamma = gamma
        self.beta = beta
    def SimplePreferenceOptimizationLoss(self,pair):
        self.preferred_reponse = pair["chosen"]
        self.rejected_response = pair["rejected"]


        self.pref_log_sft = F.log_softmax(self.policy_model(**self.preferred_reponse),dim=-1)
        self.pref_log_sft = torch.gather(self.pref_log_sft,dim=-1,index=self.preferred_reponse['input_ids'].unsqueeze(-1)).squeeze(1)
        self.pref_log_sft = self.pref_log_sft * self.preferred_response['attention_mask']
        self.pref_log_sft = torch.sum(self.pref_log_sft,dim = -1)

        self.rej_log_sft = F.log_softmax(self.policy_model(**self.rejected_reponse),dim=-1)
        self.rej_log_sft = torch.gather(self.rej_log_sft,dim=-1,index=self.rejected_response['input_ids'].unsqueeze(-1)).squeeze(1)
        self.rej_log_sft = self.rej_log_sft * self.rejected_response['attention_mask']
        self.rej_log_sft = torch.sum(self.win_log_sft,dim = -1)



        self.norm_contraint_pref = self.pref_log_sft / torch.sum(self.preferred_reponse["attention_mask"],dim=-1)
        self.norm_contraint_rej = self.rej_log_sft / torch.sum(self.rejected_response["attention_mask"],dim=-1)

        self.diff = self.norm_contraint_pref - self.norm_contraint_rej

        loss_func = -F.logsigmoid(
                self.beta(self.diff) - self.gamma
        )
        return loss_func




In [ ]:
def SPO_COLLATE(batch):
    mereged_preffered_responses = []
    mereged_rejected_responses = []

    for pair in batch:
        preferred_prompt_response = "Instruction :" + pair["instruction"] + "\n" + "Output:" + pair["chosen_response"] 
        rejected_prompt_response = "Instruction :" + pair["instruction"] + "\n" + "Output:" + pair["rejected_response"] 



        mereged_rejected_responses.append(rejected_prompt_response)
        mereged_preffered_responses.append(preferred_prompt_response)


        tokenized_preffered_response_with_prompt = tokenizer(mereged_preffered_responses,max_length = 1024,padding = "longest",trunation = True,return_tensors = "pt").to(device)
        tokenized_rejected_response_with_prompt = tokenizer(mereged_rejected_responses,max_length = 1024,padding = "longest",trunation = True,return_tensors = "pt").to(device)



        return {
            "preffered_tokens" : tokenized_preffered_response_with_prompt,
            "rejeted_tokens" : tokenized_rejected_response_with_prompt
        }


In [4]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(batch_size=batch_size,collate_fn=SPO_COLLATE,shuffle=True)
val_loader = DataLoader(batch_size=batch_size,collate_fn=SPO_COLLATE,shuffle=True)

NameError: name 'batch_size' is not defined

In [8]:
policy_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [38]:
# policy_model.eval()
prompt = "Explain the concept of reinforcement learning in simple words."

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
with torch.no_grad():
    outputs = policy_model(
        **inputs
        
    )



In [39]:
type(outputs)

transformers.modeling_outputs.CausalLMOutputWithPast

In [51]:
torch.softmax(outputs.logits,dim = -1)

tensor([[[6.0498e-06, 2.2551e-06, 1.2591e-06,  ..., 8.0986e-09,
          8.1163e-09, 8.0831e-09],
         [6.4469e-08, 1.5519e-07, 3.9755e-09,  ..., 6.6247e-11,
          6.6269e-11, 6.6318e-11],
         [2.7299e-08, 1.0857e-06, 1.0422e-08,  ..., 3.1422e-10,
          3.1385e-10, 3.1436e-10],
         ...,
         [9.5905e-07, 2.5847e-08, 8.0646e-09,  ..., 1.6673e-12,
          1.6654e-12, 1.6668e-12],
         [3.4998e-04, 5.9411e-06, 1.9451e-06,  ..., 5.5332e-11,
          5.5433e-11, 5.5435e-11],
         [2.0611e-07, 8.0198e-10, 5.2417e-09,  ..., 1.1033e-11,
          1.1041e-11, 1.1044e-11]]])

In [49]:
outputs.logits.shape

torch.Size([1, 11, 151936])

In [58]:
inputs['input_ids'].unsqueeze(-1).shape

torch.Size([1, 11, 1])

In [61]:
inputs['attention_mask'].shape

torch.Size([1, 11])

In [65]:
##loading datasets
from datasets import load_dataset, Dataset

train_dataset = load_dataset("argilla/ultrafeedback-binarized-preferences", split="train")


README.md:   0%|          | 0.00/8.62k [00:00<?, ?B/s]

(…)-00000-of-00001-9dffc9d46d32c335.parquet:   0%|          | 0.00/110M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/63619 [00:00<?, ? examples/s]

In [67]:
train_val_split = train_dataset.train_test_split(test_size=0.2, seed=42)

In [70]:
train_dataset = train_val_split['train']
test_dataset = train_val_split['test']

In [71]:
train_dataset

Dataset({
    features: ['source', 'instruction', 'chosen_response', 'rejected_response', 'chosen_avg_rating', 'rejected_avg_rating', 'chosen_model'],
    num_rows: 50895
})

In [72]:
test_dataset

Dataset({
    features: ['source', 'instruction', 'chosen_response', 'rejected_response', 'chosen_avg_rating', 'rejected_avg_rating', 'chosen_model'],
    num_rows: 12724
})